In [1]:
import sys
# sys.path.append('/Users/joachim/texjs/lva/IntroSC/ASC-ODE/build/mechsystem')
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

In [2]:
import numpy as np
from mass_spring import *

mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)

R = 0.5
H = 1.0
OMEGA = 20.0

f1 = mss.add(Fix((0.0, 0.0, 0.0)))

m_mass = 1.0
phi = [0.0, 2*np.pi/3, 4*np.pi/3]

def pos(phi):
    return np.array([R*np.cos(phi), R*np.sin(phi), H], dtype=float)

def vel_from_pos(p):
    x, y, z = p
    return (-OMEGA * y, OMEGA * x, 0.0)  

pA, pB, pC = pos(phi[0]), pos(phi[1]), pos(phi[2])

mA = mss.add(Mass(m_mass, tuple(pA), vel_from_pos(pA)))
mB = mss.add(Mass(m_mass, tuple(pB), vel_from_pos(pB)))
mC = mss.add(Mass(m_mass, tuple(pC), vel_from_pos(pC)))

L_TIP  = float(np.linalg.norm(pA - np.array([0.0,0.0,0.0]))) 
L_BASE = float(np.linalg.norm(pA - pB))                      

mss.add(DistanceConstraint(f1, mA, L_TIP))
mss.add(DistanceConstraint(f1, mB, L_TIP))
mss.add(DistanceConstraint(f1, mC, L_TIP))

mss.add(DistanceConstraint(mA, mB, L_BASE))
mss.add(DistanceConstraint(mB, mC, L_BASE))
mss.add(DistanceConstraint(mC, mA, L_BASE))

print("masses:", len(mss.masses), "joints:", len(mss.joints))


masses: 3 joints: 6


In [3]:
from pythreejs import *

masses = []
for m in mss.masses:
    masses.append(
        Mesh(
            SphereBufferGeometry(0.2, 16, 16),
            MeshStandardMaterial(color='red'),
            position=(m.pos[0], m.pos[1], m.pos[2]),
        )
    )

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(
            SphereBufferGeometry(0.2, 32, 16),
            MeshStandardMaterial(color='blue'),
            position=(f.pos[0], f.pos[1], f.pos[2]),
        )
    )

constraint_pos = []
for c in mss.joints: 
    pA = mss[c.connectors[0]].pos
    pB = mss[c.connectors[1]].pos
    constraint_pos.append(
        [[pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]]]
    )

constraint_geo = LineSegmentsGeometry(positions=constraint_pos)
constraint_mat = LineMaterial(linewidth=3, color='cyan')
constraints_line = LineSegments2(constraint_geo, constraint_mat)

axes = AxesHelper(1)


In [4]:
view_width = 600
view_height = 400

camera = PerspectiveCamera(position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

children = [*masses, *fixes, axes, camera, key_light, ambient_light]

if "constraints_line" in globals() and constraints_line is not None:
    children.insert(2, constraints_line)

scene = Scene(children=children)
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer


Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [5]:
from time import sleep
from pythreejs import LineSegmentsGeometry

def get_constraint_segments():
    segs = []
    for c in mss.joints: 
        pA = mss[c.connectors[0]].pos
        pB = mss[c.connectors[1]].pos
        segs.append([[pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]]])
    return segs

for i in range(5000):
    mss.simulate(0.01, 1)

    for m, mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    if "constraints_line" in globals() and constraints_line is not None:
        segs = get_constraint_segments()
        if segs:
            constraints_line.geometry = LineSegmentsGeometry(positions=segs)

    sleep(0.02)


KeyboardInterrupt: 